In [1]:
from ai import dprint
from copy import deepcopy
from random import choice

def ai_by_score(mb_orig, eval_func, debug=False, rand=True, candidate=False):   
    dprint(debug, "Start ai_by_score")
    dprint(debug, mb_orig)
    legal_moves = mb_orig.calc_legal_moves()
    dprint(debug, "legal_moves", legal_moves)
    best_score = float("-inf")
    best_moves = []
    for move in legal_moves:
        dprint(debug, "=" * 20)
        dprint(debug, "move", move)
        mb = deepcopy(mb_orig)
        x, y = move
        mb.move(x, y)
        dprint(debug, mb)
        
        score = eval_func(mb)
        dprint(debug, "score", score, "best score", best_score)
        
        if best_score < score:
            best_score = score
            best_moves = [move]
            dprint(debug, "UPDATE")
            dprint(debug, "  best score", best_score)
            dprint(debug, "  best moves", best_moves)
        elif best_score == score:
            best_moves.append(move)
            dprint(debug, "APPEND")
            dprint(debug, "  best moves", best_moves)

    dprint(debug, "=" * 20)
    dprint(debug, "Finished")
    dprint(debug, "best score", best_score)
    dprint(debug, "best moves", best_moves)
    if candidate:
        return best_moves
    elif rand:   
        return choice(best_moves)
    else:
        return best_moves[0]

In [2]:
def ai2s(mb, debug=False, candidate=False):
    def eval_func(mb):
        return 0
        
    return ai_by_score(mb, eval_func, debug=debug, candidate=candidate)

In [3]:
from marubatsu import Marubatsu

mb = Marubatsu()
ai2s(mb, candidate=True)


[(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)]

In [4]:
from marubatsu import Markpat

def ai14s(mb, score_victory=300, score_sure_victory=200, score_defeat=-100, score_special=100, \
          score_201=2, score_102=0.5, score_012=-1, debug=False, candidate=False):       
    def eval_func(mb):         
        # 評価値の合計を計算する変数を 0 で初期化する
        score = 0        

        # 自分が勝利している場合
        if mb.status == mb.last_turn:
            return score_victory

        markpats = mb.count_markpats()
        if debug:
            pprint(markpats)
        # 相手が勝利できる場合は評価値を加算する
        if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
            score = score_defeat * markpats[Markpat(last_turn=0, turn=2, empty=1)]
        # 次の自分の手番で自分が必ず勝利できる場合
        elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
            return score_sure_victory
        
        # 斜め方向に 〇×〇 が並び、いずれかの辺の 1 つのマスのみに × が配置されている場合
        if mb.board[1][1] == Marubatsu.CROSS and \
           (mb.board[0][0] == mb.board[2][2] == Marubatsu.CIRCLE or \
            mb.board[2][0] == mb.board[0][2] == Marubatsu.CIRCLE) and \
           (mb.board[1][0] == Marubatsu.CROSS or \
            mb.board[0][1] == Marubatsu.CROSS or \
            mb.board[2][1] == Marubatsu.CROSS or \
            mb.board[1][2] == Marubatsu.CROSS) and \
           mb.move_count == 4:
            return score_special    

        # 次の自分の手番で自分が勝利できる場合は評価値に score_201 を加算する
        if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
            score += score_201
        # 「自 1 敵 0 空 2」1 つあたり score_102 だけ、評価値を加算する
        score += markpats[Markpat(last_turn=1, turn=0, empty=2)] * score_102
        # 「自 0 敵 1 空 2」1 つあたり score_201 だけ、評価値を減算する
        score += markpats[Markpat(last_turn=0, turn=1, empty=2)] * score_012
        
        # 計算した評価値を返す
        return score

    return ai_by_score(mb, eval_func, debug=debug, candidate=candidate)

In [5]:
mb.move(1, 1)
ai14s(mb, candidate=True)

[(0, 0), (2, 0), (0, 2), (2, 2)]

In [6]:
import pickle
import gzip

def load_bestmoves(fname="../data/bestmoves.dat"):
    with gzip.open(fname, "rb") as f:
        return pickle.load(f)

In [7]:
bestmoves = load_bestmoves()

In [8]:
print(mb)

Turn x
...
.O.
...



In [9]:
bestmove = bestmoves[tuple(mb.records)]
print(bestmove)

[(0, 0), (2, 0), (0, 2), (2, 2)]


In [10]:
moves_ai2s = ai2s(mb, candidate=True)
moves_ai14s = ai14s(mb, candidate=True)
print(moves_ai2s)
print(moves_ai14s)

[(0, 0), (1, 0), (2, 0), (0, 1), (2, 1), (0, 2), (1, 2), (2, 2)]
[(0, 0), (2, 0), (0, 2), (2, 2)]


In [11]:
print(set(moves_ai2s))

{(0, 1), (1, 2), (2, 1), (0, 0), (2, 0), (0, 2), (2, 2), (1, 0)}


In [12]:
print(set(moves_ai2s) <= set(bestmove))
print(set(moves_ai14s) <= set(bestmove))

False
True


In [16]:
print(set(moves_ai2s) > set(bestmove))
print(set(moves_ai14s) > set(bestmove))

True
False


In [13]:
from tree import Mbtree

mbtree = Mbtree.load("../data/aidata")
mblist = []
for node in mbtree.nodelist:
    if node.mb.status == Marubatsu.PLAYING:
        mblist.append(node.mb)

In [14]:
with gzip.open("../data/mblist.dat", "wb") as f:
    pickle.dump(mblist, f)

In [15]:
def load_mblist(fname="../data/mblist.dat"):
    with gzip.open(fname, "rb") as f:
        return pickle.load(f)

In [17]:
from tqdm import tqdm

def is_strongly_solved(ai):
    bestmoves = load_bestmoves()
    mblist = load_mblist()
    for mb in tqdm(mblist):
        if set(ai(mb, candidate=True)) > set(bestmoves[tuple(mb.records)]):
            return False
    return True

In [18]:
is_strongly_solved(ai2s)

  0%|          | 1/294778 [00:00<?, ?it/s]


False

In [19]:
is_strongly_solved(ai14s)

100%|██████████| 294778/294778 [00:53<00:00, 5487.84it/s]


True

In [20]:
class Check_solved:
    bestmoves = load_bestmoves()
    mblist = load_mblist()
    
    @staticmethod
    def is_strongly_solved(ai):
        for mb in tqdm(Check_solved.mblist):
            if set(ai(mb, candidate=True)) > set(Check_solved.bestmoves[tuple(mb.records)]):
                return False
        return True

In [21]:
Check_solved.is_strongly_solved(ai14s)

100%|██████████| 294778/294778 [00:53<00:00, 5468.21it/s]


True

In [22]:
@staticmethod
def is_strongly_solved(ai):
    count = 0
    for mb in tqdm(Check_solved.mblist):
        if set(ai(mb, candidate=True)) <= set(Check_solved.bestmoves[tuple(mb.records)]):
            count += 1
    nodenum = len(Check_solved.mblist)
    print(f"{count}/{nodenum} {count/nodenum*100:.2f}%")
    return count == nodenum

Check_solved.is_strongly_solved = is_strongly_solved

In [23]:
Check_solved.is_strongly_solved(ai2s)

100%|██████████| 294778/294778 [00:43<00:00, 6851.92it/s]

173985/294778 59.02%


False